In [4]:
import pandas as pd

data = pd.read_csv("../../data/cleaned_data.csv")
data.head()

,text,label
0,នាយិកា មជ្ឈមណ្ឌល សិទ្ធិ មនុស្ស កម្ពុជា អ្នកស្រ...,neutral
1,ការឃុំ កញ្ញា សេង ធារី កាន់តែ យូរ រដ្ឋាភិបាល ហ៊...,positive
2,ប្រភព បង្ហើប បន្ទប់ ខ្ទង់ ចំណាយ ជាង ១០ម៉ឺន ដុល...,neutral
3,1956 បាន បង្ហាញ ផូស្វ័រ បាន ផ្ទេរ ដើម បែក អារ ...,neutral
4,ដរាបណា មិន បាន តាំងចិត្ត ខិតខំ ប្រឹង រៀន ប្រឹង...,negative


In [5]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
# Features and labels
X = data['text']  # cleaned text
y = data['label']      # labels

# Split dataset: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # stratify keeps class distribution
)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Vectorize text
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(X_train)  # X_texts = list of Khmer sentences

# 2. Oversample
ros = RandomOverSampler(random_state=42)
# Convert sparse matrix to dense temporarily
X_dense = X_vec.toarray()
X_res, y_res = ros.fit_resample(X_dense, y_train)

# 3. Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42, stratify=y_res)

# 4. Define models
models = {
    "MultinomialNB": MultinomialNB(),
    "LogisticRegression": LogisticRegression(max_iter=500, class_weight='balanced'),
    "LinearSVC": LinearSVC(class_weight='balanced', max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    "DecisionTree": DecisionTreeClassifier(class_weight='balanced', random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, random_state=42)
}

# 5. Train & evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"=== {name} ===")
    print("Accuracy:", acc)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("Confusion Matrix:")
    print(cm)

    print("-" * 50)

=== MultinomialNB ===
Accuracy: 0.6429592566118656

Classification Report:
              precision    recall  f1-score   support

    negative       0.58      0.69      0.63       932
     neutral       0.73      0.65      0.69       933
    positive       0.64      0.58      0.61       933

    accuracy                           0.64      2798
   macro avg       0.65      0.64      0.64      2798
weighted avg       0.65      0.64      0.64      2798

Confusion Matrix:
[[646 108 178]
 [195 609 129]
 [275 114 544]]
--------------------------------------------------
=== LogisticRegression ===
Accuracy: 0.7112223016440314

Classification Report:
              precision    recall  f1-score   support

    negative       0.67      0.67      0.67       932
     neutral       0.73      0.90      0.80       933
    positive       0.74      0.57      0.64       933

    accuracy                           0.71      2798
   macro avg       0.71      0.71      0.70      2798
weighted avg       0.71

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

results = []

for name, model in models.items():
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average="macro")

    results.append({
        "model": name,
        "accuracy": acc,
        "macro_f1": macro_f1
    })

df_results = pd.DataFrame(results)
print(df_results.sort_values("macro_f1", ascending=False))


In [ ]:
best_overall = df_results.loc[df_results["macro_f1"].idxmax()]

print("🏆 Best model overall")
print(best_overall)
